# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Environment, ScriptRunConfig
Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

{
    "assetId": null,
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230509.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "sklearn-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
     

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()
experiment_name = 'heart_failure_prediction_hyperml4'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')


Workspace name: quick-starts-ws-251683
Azure region: westeurope
Subscription id: 9b72f9e6-56c5-4c16-991b-19c652994860
Resource group: aml-quickstarts-251683


In [4]:

param_sampling = RandomParameterSampling(
    { 
        '--C': choice(0.1, 1.0,10),
        '--max_iter': choice(['5', '10', '20'])
    }
)

early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1, delay_evaluation=0)
#policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)



# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')
cluster_name = "optCompute"
src = ScriptRunConfig(source_directory='./',
                                script='train.py',
                                environment=sklearn_env, 
                                compute_target=cluster_name)


hyperdrive_config  = HyperDriveConfig(hyperparameter_sampling = param_sampling , primary_metric_name = "Accuracy", primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, max_total_runs=20, max_concurrent_runs=2, max_duration_minutes=30, policy=early_termination_policy, estimator=None, run_config=src, resume_from=None, resume_child_runs=None, pipeline=None, debug_flag=None, custom_run_id=None)

hdr= experiment.submit(config=hyperdrive_config)



## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_9df988e3-3199-4212-972b-7ed88338a542
Web View: https://ml.azure.com/runs/HD_9df988e3-3199-4212-972b-7ed88338a542?wsid=/subscriptions/9b72f9e6-56c5-4c16-991b-19c652994860/resourcegroups/aml-quickstarts-251683/workspaces/quick-starts-ws-251683&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-02-05T01:41:54.086144][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space
[2024-02-05T01:41:54.6121016Z][SCHEDULER][INFO]Scheduling job, id='HD_9df988e3-3199-4212-972b-7ed88338a542_0' 
[2024-02-05T01:41:54.7372330Z][SCHEDULER][INFO]Scheduling job, id='HD_9df988e3-3199-4212-972b-7ed88338a542_1' 
[2024-02-05T01:41:54.675649][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.
[2024-02-05T01:41:55.0129177Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_9df988e3-3199-4212-972b-7ed88338a542_0' 
[2024-02-05T01:41:55.0041968Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_9df988

{'runId': 'HD_9df988e3-3199-4212-972b-7ed88338a542',
 'target': 'optCompute',
 'status': 'Completed',
 'startTimeUtc': '2024-02-05T01:41:53.260005Z',
 'endTimeUtc': '2024-02-05T02:01:02.663337Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '90dad892-14ac-4d78-aa5f-d7f8920c22a7',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '9',
  'best_child_run_id': 'HD_9df988e3-3199-4212-972b-7ed88338a542_7',
  'score': '0.7207792207792207',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_9df988e3-3199-4212-972b-7ed88338a542_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientTyp

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [6]:
import joblib
from azureml.train.hyperdrive.run import HyperDriveRun
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = HyperDriveRun.get_best_run_by_primary_metric(hdr)
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']#['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
#print('\n learning rate:',parameter_values[3])
#print('\n keep probability:',parameter_values[5])
#print('\n batch size:',parameter_values[7])

best_run.get_file_names()

Best Run Id:  HD_9df988e3-3199-4212-972b-7ed88338a542_7

 Accuracy: 0.7207792207792207


['system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [7]:
#TODO: Save the best model

best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])
model = best_run.register_model(model_name='HyperDrive_HighAccuracy', model_path='outputs/', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                           'Max Iterations': best_run_metrics['Max iterations:']})

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs in the set of files uploaded to the run: ['system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path outputs in the set of files uploaded to the run: ['system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

